# Per-Target Ensemble Weight Optimization

**Hypothesis**: Different targets (SM, Product 2, Product 3) may benefit from different ensemble weights.

**Current weights**: GP(0.15) + MLP(0.55) + LGBM(0.30)

**Implementation**:
1. Train GP, MLP, LGBM separately
2. Optimize weights for each target using grid search
3. Use target-specific weights in final ensemble

In [1]:
# Standard imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import lightgbm as lgb
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel
from sklearn.preprocessing import StandardScaler
from itertools import product
import tqdm
import warnings
warnings.filterwarnings('ignore')

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.set_default_dtype(torch.double)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [2]:
# Data loading functions
DATA_PATH = '/home/data'

INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_FULL_SOLVENT = ["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

print('Data loading functions defined')

Data loading functions defined


In [3]:
# Load feature lookups
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
DRFP_DF = pd.read_csv(f'{DATA_PATH}/drfps_catechol_lookup.csv', index_col=0)
ACS_PCA_DF = pd.read_csv(f'{DATA_PATH}/acs_pca_descriptors_lookup.csv', index_col=0)

# Filter DRFP to high-variance columns
drfp_variance = DRFP_DF.var()
nonzero_variance_cols = drfp_variance[drfp_variance > 0].index.tolist()
DRFP_FILTERED = DRFP_DF[nonzero_variance_cols]

print(f'Spange: {SPANGE_DF.shape}, DRFP filtered: {DRFP_FILTERED.shape}, ACS PCA: {ACS_PCA_DF.shape}')

Spange: (26, 13), DRFP filtered: (24, 122), ACS PCA: (24, 5)


In [4]:
# Full Featurizer - 145 features
class FullFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.drfp_df = DRFP_FILTERED
        self.acs_pca_df = ACS_PCA_DF
        self.feats_dim = 2 + 3 + self.spange_df.shape[1] + self.drfp_df.shape[1] + self.acs_pca_df.shape[1]

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            A_drfp = self.drfp_df.loc[X["SOLVENT A NAME"]].values
            B_drfp = self.drfp_df.loc[X["SOLVENT B NAME"]].values
            A_acs = self.acs_pca_df.loc[X["SOLVENT A NAME"]].values
            B_acs = self.acs_pca_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
                X_drfp = B_drfp * (1 - (1-pct)) + A_drfp * (1-pct)
                X_acs = B_acs * (1 - (1-pct)) + A_acs * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
                X_drfp = A_drfp * (1 - pct) + B_drfp * pct
                X_acs = A_acs * (1 - pct) + B_acs * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
            X_drfp = self.drfp_df.loc[X["SOLVENT NAME"]].values
            X_acs = self.acs_pca_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange, X_drfp, X_acs])

# Simple Featurizer (for GP) - 18 features
class SimpleFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.feats_dim = 2 + 3 + self.spange_df.shape[1]

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange])

print(f'Full feature dimension: {FullFeaturizer().feats_dim}')
print(f'Simple feature dimension: {SimpleFeaturizer().feats_dim}')

Full feature dimension: 145
Simple feature dimension: 18


In [5]:
# MLP Model
class MLPModel(nn.Module):
    def __init__(self, input_dim=145, hidden_dim=64, output_dim=3, dropout=0.2):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim // 2)
        self.fc3 = nn.Linear(hidden_dim // 2, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = torch.sigmoid(self.fc3(x))
        return x

print('MLP model defined')

MLP model defined


In [6]:
# Per-Target Weight Ensemble Model
class PerTargetWeightEnsemble:
    """Ensemble with different weights for each target."""
    def __init__(self, data='single', 
                 weights_target0=(0.15, 0.55, 0.30),  # Product 2
                 weights_target1=(0.15, 0.55, 0.30),  # Product 3
                 weights_target2=(0.15, 0.55, 0.30)): # SM
        self.data = data
        self.weights = [weights_target0, weights_target1, weights_target2]
        self.mixed = (data == 'full')
        
        self.full_featurizer = FullFeaturizer(mixed=self.mixed)
        self.simple_featurizer = SimpleFeaturizer(mixed=self.mixed)
        
        self.scaler_full = StandardScaler()
        self.scaler_simple = StandardScaler()
        self.gp_models = []
        self.mlp_model = None
        self.lgbm_models = []
        
    def train_model(self, X, Y):
        X_full = self.full_featurizer.featurize(X)
        X_simple = self.simple_featurizer.featurize(X)
        Y_np = Y.values
        
        X_full_scaled = self.scaler_full.fit_transform(X_full)
        X_simple_scaled = self.scaler_simple.fit_transform(X_simple)
        
        # Train GP for each target
        for i in range(3):
            kernel = ConstantKernel(1.0) * Matern(length_scale=1.0, nu=2.5) + WhiteKernel(noise_level=0.1)
            gp = GaussianProcessRegressor(
                kernel=kernel, alpha=0.01, n_restarts_optimizer=3, random_state=42
            )
            gp.fit(X_simple_scaled, Y_np[:, i])
            self.gp_models.append(gp)
        
        # Train MLP
        self.mlp_model = MLPModel(
            input_dim=self.full_featurizer.feats_dim,
            hidden_dim=64,
            output_dim=3,
            dropout=0.2
        ).to(device)
        
        X_tensor = torch.tensor(X_full_scaled, dtype=torch.double).to(device)
        Y_tensor = torch.tensor(Y_np, dtype=torch.double).to(device)
        
        optimizer = torch.optim.Adam(self.mlp_model.parameters(), lr=0.001, weight_decay=1e-4)
        criterion = nn.MSELoss()
        
        self.mlp_model.train()
        for epoch in range(200):
            optimizer.zero_grad()
            pred = self.mlp_model(X_tensor)
            loss = criterion(pred, Y_tensor)
            loss.backward()
            optimizer.step()
        
        # Train LGBM for each target
        for i in range(3):
            lgbm = lgb.LGBMRegressor(
                n_estimators=100, learning_rate=0.05, max_depth=5,
                num_leaves=31, min_child_samples=5, random_state=42, verbose=-1
            )
            lgbm.fit(X_full_scaled, Y_np[:, i])
            self.lgbm_models.append(lgbm)
        
        return self
    
    def predict(self, X):
        X_full = self.full_featurizer.featurize(X)
        X_simple = self.simple_featurizer.featurize(X)
        
        X_full_scaled = self.scaler_full.transform(X_full)
        X_simple_scaled = self.scaler_simple.transform(X_simple)
        
        # GP predictions
        gp_preds = np.column_stack([self.gp_models[i].predict(X_simple_scaled) for i in range(3)])
        
        # MLP predictions
        self.mlp_model.eval()
        X_tensor = torch.tensor(X_full_scaled, dtype=torch.double).to(device)
        with torch.no_grad():
            mlp_preds = self.mlp_model(X_tensor).cpu().numpy()
        
        # LGBM predictions
        lgbm_preds = np.column_stack([self.lgbm_models[i].predict(X_full_scaled) for i in range(3)])
        
        # Per-target weighted ensemble
        predictions = np.zeros_like(gp_preds)
        for i in range(3):
            gp_w, mlp_w, lgbm_w = self.weights[i]
            predictions[:, i] = gp_w * gp_preds[:, i] + mlp_w * mlp_preds[:, i] + lgbm_w * lgbm_preds[:, i]
        
        # TTA for mixtures
        if self.mixed:
            X_full_flip = self.full_featurizer.featurize(X, flip=True)
            X_simple_flip = self.simple_featurizer.featurize(X, flip=True)
            
            X_full_scaled_flip = self.scaler_full.transform(X_full_flip)
            X_simple_scaled_flip = self.scaler_simple.transform(X_simple_flip)
            
            gp_preds_flip = np.column_stack([self.gp_models[i].predict(X_simple_scaled_flip) for i in range(3)])
            X_tensor_flip = torch.tensor(X_full_scaled_flip, dtype=torch.double).to(device)
            with torch.no_grad():
                mlp_preds_flip = self.mlp_model(X_tensor_flip).cpu().numpy()
            lgbm_preds_flip = np.column_stack([self.lgbm_models[i].predict(X_full_scaled_flip) for i in range(3)])
            
            predictions_flip = np.zeros_like(gp_preds_flip)
            for i in range(3):
                gp_w, mlp_w, lgbm_w = self.weights[i]
                predictions_flip[:, i] = gp_w * gp_preds_flip[:, i] + mlp_w * mlp_preds_flip[:, i] + lgbm_w * lgbm_preds_flip[:, i]
            
            predictions = (predictions + predictions_flip) / 2
        
        predictions = np.clip(predictions, 0, 1)
        return torch.tensor(predictions)

print('Per-Target Weight Ensemble defined')

Per-Target Weight Ensemble defined


In [7]:
# First, let's collect individual model predictions to find optimal weights
# Run CV and store GP, MLP, LGBM predictions separately

print('=== Collecting Individual Model Predictions for Weight Optimization ===')

X_single, Y_single = load_data("single_solvent")

# Storage for predictions
all_gp_preds = []
all_mlp_preds = []
all_lgbm_preds = []
all_actuals = []

split_generator = generate_leave_one_out_splits(X_single, Y_single)

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=24):
    (train_X, train_Y), (test_X, test_Y) = split
    
    # Featurize
    full_feat = FullFeaturizer(mixed=False)
    simple_feat = SimpleFeaturizer(mixed=False)
    
    X_train_full = full_feat.featurize(train_X)
    X_test_full = full_feat.featurize(test_X)
    X_train_simple = simple_feat.featurize(train_X)
    X_test_simple = simple_feat.featurize(test_X)
    Y_train = train_Y.values
    Y_test = test_Y.values
    
    # Scale
    scaler_full = StandardScaler()
    scaler_simple = StandardScaler()
    X_train_full_scaled = scaler_full.fit_transform(X_train_full)
    X_test_full_scaled = scaler_full.transform(X_test_full)
    X_train_simple_scaled = scaler_simple.fit_transform(X_train_simple)
    X_test_simple_scaled = scaler_simple.transform(X_test_simple)
    
    # Train and predict with GP
    gp_preds = []
    for i in range(3):
        kernel = ConstantKernel(1.0) * Matern(length_scale=1.0, nu=2.5) + WhiteKernel(noise_level=0.1)
        gp = GaussianProcessRegressor(kernel=kernel, alpha=0.01, n_restarts_optimizer=3, random_state=42)
        gp.fit(X_train_simple_scaled, Y_train[:, i])
        gp_preds.append(gp.predict(X_test_simple_scaled))
    gp_preds = np.column_stack(gp_preds)
    
    # Train and predict with MLP
    mlp = MLPModel(input_dim=full_feat.feats_dim, hidden_dim=64, output_dim=3, dropout=0.2).to(device)
    X_tensor = torch.tensor(X_train_full_scaled, dtype=torch.double).to(device)
    Y_tensor = torch.tensor(Y_train, dtype=torch.double).to(device)
    optimizer = torch.optim.Adam(mlp.parameters(), lr=0.001, weight_decay=1e-4)
    criterion = nn.MSELoss()
    mlp.train()
    for epoch in range(200):
        optimizer.zero_grad()
        pred = mlp(X_tensor)
        loss = criterion(pred, Y_tensor)
        loss.backward()
        optimizer.step()
    mlp.eval()
    X_test_tensor = torch.tensor(X_test_full_scaled, dtype=torch.double).to(device)
    with torch.no_grad():
        mlp_preds = mlp(X_test_tensor).cpu().numpy()
    
    # Train and predict with LGBM
    lgbm_preds = []
    for i in range(3):
        lgbm = lgb.LGBMRegressor(n_estimators=100, learning_rate=0.05, max_depth=5,
                                  num_leaves=31, min_child_samples=5, random_state=42, verbose=-1)
        lgbm.fit(X_train_full_scaled, Y_train[:, i])
        lgbm_preds.append(lgbm.predict(X_test_full_scaled))
    lgbm_preds = np.column_stack(lgbm_preds)
    
    all_gp_preds.append(gp_preds)
    all_mlp_preds.append(mlp_preds)
    all_lgbm_preds.append(lgbm_preds)
    all_actuals.append(Y_test)

# Stack all predictions
gp_all = np.vstack(all_gp_preds)
mlp_all = np.vstack(all_mlp_preds)
lgbm_all = np.vstack(all_lgbm_preds)
actuals_all = np.vstack(all_actuals)

print(f'\nCollected predictions: GP={gp_all.shape}, MLP={mlp_all.shape}, LGBM={lgbm_all.shape}')

=== Collecting Individual Model Predictions for Weight Optimization ===


  0%|          | 0/24 [00:00<?, ?it/s]

  4%|▍         | 1/24 [00:30<11:33, 30.13s/it]

  8%|▊         | 2/24 [00:57<10:32, 28.77s/it]

 12%|█▎        | 3/24 [01:23<09:34, 27.36s/it]

...

 92%|█████████▏| 22/24 [10:31<00:57, 28.54s/it]

 96%|█████████▌| 23/24 [11:02<00:29, 29.45s/it]

100%|██████████| 24/24 [11:30<00:00, 28.97s/it]

100%|██████████| 24/24 [11:30<00:00, 28.78s/it]


Collected predictions: GP=(656, 3), MLP=(656, 3), LGBM=(656, 3)


In [8]:
# Grid search for optimal weights per target
print('\n=== Grid Search for Optimal Per-Target Weights ===')

weight_options = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

best_weights = []
best_mses = []

for target_idx in range(3):
    target_names = ['Product 2', 'Product 3', 'SM']
    print(f'\nOptimizing weights for {target_names[target_idx]}...')
    
    gp_pred = gp_all[:, target_idx]
    mlp_pred = mlp_all[:, target_idx]
    lgbm_pred = lgbm_all[:, target_idx]
    actual = actuals_all[:, target_idx]
    
    best_mse = float('inf')
    best_w = None
    
    for gp_w in weight_options:
        for mlp_w in weight_options:
            lgbm_w = 1.0 - gp_w - mlp_w
            if lgbm_w < 0 or lgbm_w > 1:
                continue
            
            ensemble_pred = gp_w * gp_pred + mlp_w * mlp_pred + lgbm_w * lgbm_pred
            ensemble_pred = np.clip(ensemble_pred, 0, 1)
            mse = np.mean((ensemble_pred - actual) ** 2)
            
            if mse < best_mse:
                best_mse = mse
                best_w = (gp_w, mlp_w, lgbm_w)
    
    best_weights.append(best_w)
    best_mses.append(best_mse)
    print(f'  Best weights: GP={best_w[0]:.1f}, MLP={best_w[1]:.1f}, LGBM={best_w[2]:.1f}')
    print(f'  Best MSE: {best_mse:.6f}')

print(f'\n=== Summary ===')
print(f'Product 2: GP={best_weights[0][0]:.1f}, MLP={best_weights[0][1]:.1f}, LGBM={best_weights[0][2]:.1f} (MSE={best_mses[0]:.6f})')
print(f'Product 3: GP={best_weights[1][0]:.1f}, MLP={best_weights[1][1]:.1f}, LGBM={best_weights[1][2]:.1f} (MSE={best_mses[1]:.6f})')
print(f'SM:        GP={best_weights[2][0]:.1f}, MLP={best_weights[2][1]:.1f}, LGBM={best_weights[2][2]:.1f} (MSE={best_mses[2]:.6f})')

# Calculate overall MSE with optimal weights
overall_mse_optimal = np.mean(best_mses)
print(f'\nOverall MSE with optimal per-target weights: {overall_mse_optimal:.6f}')


=== Grid Search for Optimal Per-Target Weights ===

Optimizing weights for Product 2...
  Best weights: GP=0.5, MLP=0.5, LGBM=0.0
  Best MSE: 0.006714

Optimizing weights for Product 3...
  Best weights: GP=0.6, MLP=0.3, LGBM=0.1
  Best MSE: 0.008209

Optimizing weights for SM...
  Best weights: GP=0.4, MLP=0.4, LGBM=0.2
  Best MSE: 0.012682

=== Summary ===
Product 2: GP=0.5, MLP=0.5, LGBM=0.0 (MSE=0.006714)
Product 3: GP=0.6, MLP=0.3, LGBM=0.1 (MSE=0.008209)
SM:        GP=0.4, MLP=0.4, LGBM=0.2 (MSE=0.012682)

Overall MSE with optimal per-target weights: 0.009202


In [9]:
# Compare with baseline weights
print('\n=== Comparison with Baseline Weights ===')

# Baseline: GP=0.15, MLP=0.55, LGBM=0.30
baseline_preds = 0.15 * gp_all + 0.55 * mlp_all + 0.30 * lgbm_all
baseline_preds = np.clip(baseline_preds, 0, 1)
baseline_mse = np.mean((baseline_preds - actuals_all) ** 2)

# Optimal per-target
optimal_preds = np.zeros_like(gp_all)
for i in range(3):
    gp_w, mlp_w, lgbm_w = best_weights[i]
    optimal_preds[:, i] = gp_w * gp_all[:, i] + mlp_w * mlp_all[:, i] + lgbm_w * lgbm_all[:, i]
optimal_preds = np.clip(optimal_preds, 0, 1)
optimal_mse = np.mean((optimal_preds - actuals_all) ** 2)

print(f'Baseline weights (0.15, 0.55, 0.30) MSE: {baseline_mse:.6f}')
print(f'Optimal per-target weights MSE: {optimal_mse:.6f}')

if optimal_mse < baseline_mse:
    improvement = (baseline_mse - optimal_mse) / baseline_mse * 100
    print(f'\n✓ Per-target optimization improves by {improvement:.2f}%')
else:
    print(f'\n✗ Per-target optimization does not improve')


=== Comparison with Baseline Weights ===
Baseline weights (0.15, 0.55, 0.30) MSE: 0.009792
Optimal per-target weights MSE: 0.009202

✓ Per-target optimization improves by 6.03%


In [ ]:
# Now run full CV with optimal per-target weights
print('\n=== Full CV with Optimal Per-Target Weights ===')

# Use the optimal weights found
weights_target0 = best_weights[0]  # Product 2
weights_target1 = best_weights[1]  # Product 3
weights_target2 = best_weights[2]  # SM

print(f'Using weights:')
print(f'  Product 2: GP={weights_target0[0]:.1f}, MLP={weights_target0[1]:.1f}, LGBM={weights_target0[2]:.1f}')
print(f'  Product 3: GP={weights_target1[0]:.1f}, MLP={weights_target1[1]:.1f}, LGBM={weights_target1[2]:.1f}')
print(f'  SM:        GP={weights_target2[0]:.1f}, MLP={weights_target2[1]:.1f}, LGBM={weights_target2[2]:.1f}')